In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import json

In [2]:
# Paths
base_dir = Path("/home/diya.thakor/AirQuality/BASELINE")
data_dir = base_dir / "data"
results_dir = base_dir / "results"
results_dir.mkdir(parents=True, exist_ok=True)

print("="*80)
print("SIMPLE HEURISTIC BASELINES")
print("="*80)

# Load train and test data
print("\n[STEP 1] Loading data...")
train = pd.read_csv(data_dir / "train_2022.csv")
test = pd.read_csv(data_dir / "test_2023.csv")

print(f"✓ Training data: {len(train)} rows, {train['city'].nunique()} cities")
print(f"✓ Test data: {len(test)} rows, {test['city'].nunique()} cities")

# ============================================================================
# HEURISTIC 1: NATIONAL MONTHLY AVERAGE
# ============================================================================
print("\n" + "="*80)
print("HEURISTIC 1: NATIONAL MONTHLY AVERAGE")
print("="*80)
print("Prediction: PM2.5 = average PM2.5 for that month across all cities")

# Calculate monthly averages from training data
monthly_avg = train.groupby('month')['PM2.5'].mean().to_dict()

print("\nMonthly averages from training data:")
for month, avg in sorted(monthly_avg.items()):
    print(f"  Month {month:2d}: {avg:.2f} μg/m³")

# Predict on test set
test['pred_national_monthly'] = test['month'].map(monthly_avg)

# Calculate metrics
mae_national = np.mean(np.abs(test['PM2.5'] - test['pred_national_monthly']))
rmse_national = np.sqrt(np.mean((test['PM2.5'] - test['pred_national_monthly'])**2))
r2_national = 1 - (np.sum((test['PM2.5'] - test['pred_national_monthly'])**2) / 
                   np.sum((test['PM2.5'] - test['PM2.5'].mean())**2))

print(f"\n✓ Results on test set:")
print(f"  MAE:  {mae_national:.2f} μg/m³")
print(f"  RMSE: {rmse_national:.2f} μg/m³")
print(f"  R²:   {r2_national:.4f}")

# ============================================================================
# HEURISTIC 2: TIER-BASED AVERAGE
# ============================================================================
print("\n" + "="*80)
print("HEURISTIC 2: TIER-BASED AVERAGE")
print("="*80)
print("Prediction: PM2.5 = average PM2.5 for that city's tier")

# Calculate tier averages from training data
tier_avg = train.groupby('Tier')['PM2.5'].mean().to_dict()

print("\nTier averages from training data:")
for tier, avg in sorted(tier_avg.items()):
    print(f"  {tier}: {avg:.2f} μg/m³")

# Predict on test set
test['pred_tier'] = test['Tier'].map(tier_avg)

# Calculate metrics
mae_tier = np.mean(np.abs(test['PM2.5'] - test['pred_tier']))
rmse_tier = np.sqrt(np.mean((test['PM2.5'] - test['pred_tier'])**2))
r2_tier = 1 - (np.sum((test['PM2.5'] - test['pred_tier'])**2) / 
               np.sum((test['PM2.5'] - test['PM2.5'].mean())**2))

print(f"\n✓ Results on test set:")
print(f"  MAE:  {mae_tier:.2f} μg/m³")
print(f"  RMSE: {rmse_tier:.2f} μg/m³")
print(f"  R²:   {r2_tier:.4f}")

# ============================================================================
# HEURISTIC 3: IGP-BASED AVERAGE
# ============================================================================
print("\n" + "="*80)
print("HEURISTIC 3: IGP-BASED AVERAGE")
print("="*80)
print("Prediction: PM2.5 = average PM2.5 for IGP or non-IGP cities")

# Calculate IGP averages from training data
igp_avg = train.groupby('IGP')['PM2.5'].mean().to_dict()

print("\nIGP averages from training data:")
for igp, avg in sorted(igp_avg.items()):
    region = "IGP" if igp == 1 else "Non-IGP"
    print(f"  {region}: {avg:.2f} μg/m³")

# Predict on test set
test['pred_igp'] = test['IGP'].map(igp_avg)

# Calculate metrics
mae_igp = np.mean(np.abs(test['PM2.5'] - test['pred_igp']))
rmse_igp = np.sqrt(np.mean((test['PM2.5'] - test['pred_igp'])**2))
r2_igp = 1 - (np.sum((test['PM2.5'] - test['pred_igp'])**2) / 
              np.sum((test['PM2.5'] - test['PM2.5'].mean())**2))

print(f"\n✓ Results on test set:")
print(f"  MAE:  {mae_igp:.2f} μg/m³")
print(f"  RMSE: {rmse_igp:.2f} μg/m³")
print(f"  R²:   {r2_igp:.4f}")

# ============================================================================
# HEURISTIC 4: TIER + MONTH COMBINED
# ============================================================================
print("\n" + "="*80)
print("HEURISTIC 4: TIER + MONTH COMBINED")
print("="*80)
print("Prediction: PM2.5 = average PM2.5 for that tier in that month")

# Calculate tier-month averages from training data
tier_month_avg = train.groupby(['Tier', 'month'])['PM2.5'].mean().to_dict()

print("\nSample tier-month averages (first 6):")
for i, ((tier, month), avg) in enumerate(sorted(tier_month_avg.items())[:6]):
    print(f"  {tier}, Month {month}: {avg:.2f} μg/m³")
print("  ...")

# Predict on test set
test['tier_month_key'] = list(zip(test['Tier'], test['month']))
test['pred_tier_month'] = test['tier_month_key'].map(tier_month_avg)

# Handle missing combinations (fallback to tier average)
missing_mask = test['pred_tier_month'].isnull()
if missing_mask.sum() > 0:
    print(f"\n⚠ {missing_mask.sum()} combinations not in training, using tier average as fallback")
    test.loc[missing_mask, 'pred_tier_month'] = test.loc[missing_mask, 'Tier'].map(tier_avg)

# Calculate metrics
mae_tier_month = np.mean(np.abs(test['PM2.5'] - test['pred_tier_month']))
rmse_tier_month = np.sqrt(np.mean((test['PM2.5'] - test['pred_tier_month'])**2))
r2_tier_month = 1 - (np.sum((test['PM2.5'] - test['pred_tier_month'])**2) / 
                     np.sum((test['PM2.5'] - test['PM2.5'].mean())**2))

print(f"\n✓ Results on test set:")
print(f"  MAE:  {mae_tier_month:.2f} μg/m³")
print(f"  RMSE: {rmse_tier_month:.2f} μg/m³")
print(f"  R²:   {r2_tier_month:.4f}")

# ============================================================================
# HEURISTIC 5: STATE-BASED AVERAGE
# ============================================================================
print("\n" + "="*80)
print("HEURISTIC 5: STATE-BASED AVERAGE")
print("="*80)
print("Prediction: PM2.5 = average PM2.5 for that state")

# Calculate state averages from training data
state_avg = train.groupby('state')['PM2.5'].mean().to_dict()

print(f"\nState averages calculated for {len(state_avg)} states")
print("Sample state averages (first 5):")
for i, (state, avg) in enumerate(sorted(state_avg.items())[:5]):
    print(f"  {state.title()}: {avg:.2f} μg/m³")
print("  ...")

# Predict on test set
test['pred_state'] = test['state'].map(state_avg)

# Handle states not in training (fallback to national average)
national_avg = train['PM2.5'].mean()
missing_states = test['pred_state'].isnull().sum()
if missing_states > 0:
    print(f"\n⚠ {missing_states} rows from states not in training, using national average as fallback")
    test.loc[test['pred_state'].isnull(), 'pred_state'] = national_avg

# Calculate metrics
mae_state = np.mean(np.abs(test['PM2.5'] - test['pred_state']))
rmse_state = np.sqrt(np.mean((test['PM2.5'] - test['pred_state'])**2))
r2_state = 1 - (np.sum((test['PM2.5'] - test['pred_state'])**2) / 
                np.sum((test['PM2.5'] - test['PM2.5'].mean())**2))

print(f"\n✓ Results on test set:")
print(f"  MAE:  {mae_state:.2f} μg/m³")
print(f"  RMSE: {rmse_state:.2f} μg/m³")
print(f"  R²:   {r2_state:.4f}")

# ============================================================================
# COMPARISON TABLE
# ============================================================================
print("\n" + "="*80)
print("COMPARISON: ALL HEURISTIC BASELINES")
print("="*80)

results = {
    'National Monthly Avg': {'MAE': mae_national, 'RMSE': rmse_national, 'R²': r2_national},
    'Tier-based Avg': {'MAE': mae_tier, 'RMSE': rmse_tier, 'R²': r2_tier},
    'IGP-based Avg': {'MAE': mae_igp, 'RMSE': rmse_igp, 'R²': r2_igp},
    'Tier+Month Avg': {'MAE': mae_tier_month, 'RMSE': rmse_tier_month, 'R²': r2_tier_month},
    'State-based Avg': {'MAE': mae_state, 'RMSE': rmse_state, 'R²': r2_state}
}

print("\n{:<25} {:>12} {:>12} {:>12}".format("Heuristic", "MAE", "RMSE", "R²"))
print("-"*65)
for name, metrics in results.items():
    print("{:<25} {:>12.2f} {:>12.2f} {:>12.4f}".format(
        name, metrics['MAE'], metrics['RMSE'], metrics['R²']))

# Find best heuristic
best_heuristic = min(results.items(), key=lambda x: x[1]['MAE'])
print(f"\n✓ Best heuristic (by MAE): {best_heuristic[0]}")
print(f"  MAE: {best_heuristic[1]['MAE']:.2f} μg/m³")

# ============================================================================
# CALCULATE OVER/UNDER PREDICTION
# ============================================================================
print("\n" + "="*80)
print("OVER/UNDER PREDICTION ANALYSIS")
print("="*80)

heuristics = {
    'National Monthly': 'pred_national_monthly',
    'Tier-based': 'pred_tier',
    'IGP-based': 'pred_igp',
    'Tier+Month': 'pred_tier_month',
    'State-based': 'pred_state'
}

print("\n{:<20} {:>15} {:>15} {:>15} {:>15}".format(
    "Heuristic", "Over-pred Count", "Over-pred MAE", "Under-pred Count", "Under-pred MAE"))
print("-"*85)

over_under_stats = {}

for name, pred_col in heuristics.items():
    errors = test['PM2.5'] - test[pred_col]
    
    over_pred = errors < 0  # Predicted > Actual
    under_pred = errors > 0  # Predicted < Actual
    
    over_count = over_pred.sum()
    under_count = under_pred.sum()
    
    over_mae = np.mean(np.abs(errors[over_pred])) if over_count > 0 else 0
    under_mae = np.mean(np.abs(errors[under_pred])) if under_count > 0 else 0
    
    print("{:<20} {:>15} {:>15.2f} {:>15} {:>15.2f}".format(
        name, over_count, over_mae, under_count, under_mae))
    
    over_under_stats[name] = {
        'over_count': int(over_count),
        'over_mae': float(over_mae),
        'under_count': int(under_count),
        'under_mae': float(under_mae)
    }

# ============================================================================
# SAVE RESULTS
# ============================================================================
print("\n" + "="*80)
print("SAVING RESULTS")
print("="*80)

# Save predictions
predictions_file = results_dir / "heuristic_predictions.csv"
pred_columns = ['city', 'state', 'YearMonth', 'month', 'PM2.5', 'Tier', 'IGP',
                'pred_national_monthly', 'pred_tier', 'pred_igp', 
                'pred_tier_month', 'pred_state']
test[pred_columns].to_csv(predictions_file, index=False)
print(f"\n✓ Predictions saved: {predictions_file}")

# Save metrics
metrics_file = results_dir / "heuristic_metrics.json"
results_with_over_under = {}
for name in results.keys():
    short_name = name.replace(' Avg', '').replace('-based', '').replace('+', '_')
    results_with_over_under[name] = {
        **results[name],
        **over_under_stats.get(short_name, over_under_stats.get(name, {}))
    }

with open(metrics_file, 'w') as f:
    json.dump(results_with_over_under, f, indent=2)
print(f"✓ Metrics saved: {metrics_file}")

# Save summary report
report_file = results_dir / "heuristic_report.txt"
with open(report_file, 'w') as f:
    f.write("="*80 + "\n")
    f.write("SIMPLE HEURISTIC BASELINES - SUMMARY REPORT\n")
    f.write("="*80 + "\n\n")
    
    f.write("Overview:\n")
    f.write(f"Training data: {len(train)} rows, {train['city'].nunique()} cities\n")
    f.write(f"Test data: {len(test)} rows, {test['city'].nunique()} cities\n\n")
    
    f.write("Performance Metrics:\n")
    f.write("-"*80 + "\n")
    f.write("{:<25} {:>12} {:>12} {:>12}\n".format("Heuristic", "MAE", "RMSE", "R²"))
    f.write("-"*80 + "\n")
    for name, metrics in results.items():
        f.write("{:<25} {:>12.2f} {:>12.2f} {:>12.4f}\n".format(
            name, metrics['MAE'], metrics['RMSE'], metrics['R²']))
    
    f.write(f"\n\nBest Heuristic (by MAE): {best_heuristic[0]}\n")
    f.write(f"MAE: {best_heuristic[1]['MAE']:.2f} μg/m³\n")
    
    f.write("\n" + "="*80 + "\n")
    f.write("Interpretation:\n")
    f.write("-"*80 + "\n")
    f.write("These simple heuristics serve as sanity checks:\n")
    f.write("- Any ML model (Linear Regression, Random Forest, LLM) should beat these\n")
    f.write("- If a complex model can't beat these, it's not learning meaningful patterns\n")
    f.write(f"- Best simple heuristic achieves MAE of {best_heuristic[1]['MAE']:.2f} μg/m³\n")
    f.write("- This is the minimum bar for any sophisticated model\n")

print(f"✓ Summary report saved: {report_file}")

print("\n" + "="*80)
print("✓ ALL HEURISTIC BASELINES COMPLETED!")
print("="*80)
print(f"\nKey Takeaway: Best heuristic MAE = {best_heuristic[1]['MAE']:.2f} μg/m³")
print("Any ML model should significantly beat this to be considered useful.")

SIMPLE HEURISTIC BASELINES

[STEP 1] Loading data...
✓ Training data: 1440 rows, 120 cities
✓ Test data: 1932 rows, 160 cities

HEURISTIC 1: NATIONAL MONTHLY AVERAGE
Prediction: PM2.5 = average PM2.5 for that month across all cities

Monthly averages from training data:
  Month  1: 78.06 μg/m³
  Month  2: 63.87 μg/m³
  Month  3: 63.13 μg/m³
  Month  4: 58.28 μg/m³
  Month  5: 48.86 μg/m³
  Month  6: 40.14 μg/m³
  Month  7: 23.42 μg/m³
  Month  8: 25.01 μg/m³
  Month  9: 28.07 μg/m³
  Month 10: 51.78 μg/m³
  Month 11: 82.82 μg/m³
  Month 12: 88.42 μg/m³

✓ Results on test set:
  MAE:  21.75 μg/m³
  RMSE: 30.91 μg/m³
  R²:   0.2553

HEURISTIC 2: TIER-BASED AVERAGE
Prediction: PM2.5 = average PM2.5 for that city's tier

Tier averages from training data:
  Tier1: 41.34 μg/m³
  Tier2: 50.59 μg/m³
  Tier3: 57.27 μg/m³

✓ Results on test set:
  MAE:  27.38 μg/m³
  RMSE: 36.36 μg/m³
  R²:   -0.0304

HEURISTIC 3: IGP-BASED AVERAGE
Prediction: PM2.5 = average PM2.5 for IGP or non-IGP cities

IGP